In [4]:
from gliner import GLiNER

model_path = "../models/gliner_medium-v2.1"

/opt/conda/envs/training-pipeline/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from src.word_process.inputs_outputs_process import DataProcess

data_process = DataProcess(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
text = """
Libretto by Marius Petipa, based on the 1822 novella ``Trilby, ou Le Lutin d'Argail`` by Charles Nodier, first presented by the Ballet of the Moscow Imperial Bolshoi Theatre on January 25/February 6 (Julian/Gregorian calendar dates), 1870, in Moscow with Polina Karpakova as Trilby and Ludiia Geiten as Miranda and restaged by Petipa for the Imperial Ballet at the Imperial Bolshoi Kamenny Theatre on January 17–29, 1871 in St. Petersburg with Adèle Grantzow as Trilby and Lev Ivanov as Count Leopold.
"""

labels = ["person", "book", "location", "date", "actor", "character"]

In [7]:
inputs, raw_batch = data_process.prepare_model_inputs([text], labels)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
import torch
import onnxruntime
ort_sess = onnxruntime.InferenceSession("../models/gliner_medium-v2.1/model.onnx")

outputs = ort_sess.run(None, {'input_ids': inputs['input_ids'].numpy(),
                            'attention_mask': inputs['attention_mask'].numpy(),
                            'words_mask': inputs['words_mask'].numpy(),
                            'text_lengths': inputs['text_lengths'].numpy(),
                            'span_idx': inputs['span_idx'].numpy(),
                            'span_mask': inputs['span_mask'].numpy(),
                            })[0]
outputs = torch.from_numpy(outputs)

In [9]:
import torch
outputs = data_process.decode(raw_batch["tokens"], raw_batch["id_to_classes"], torch.tensor(outputs))

/tmp/ipykernel_106350/3381801605.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = data_process.decode(raw_batch["tokens"], raw_batch["id_to_classes"], torch.tensor(outputs))


In [10]:
outputs[0]

[(12, 12, 'character', 0.9745467305183411),
 (33, 33, 'location', 0.9113682508468628),
 (38, 42, 'date', 0.7391266226768494),
 (51, 51, 'date', 0.5650821924209595),
 (54, 54, 'location', 0.9242361187934875),
 (56, 57, 'actor', 0.9268550276756287),
 (59, 59, 'character', 0.9865744709968567),
 (61, 62, 'actor', 0.8973000645637512),
 (64, 64, 'character', 0.7989766597747803),
 (80, 85, 'date', 0.9063624143600464),
 (87, 89, 'location', 0.9150590300559998),
 (91, 92, 'actor', 0.9332297444343567),
 (94, 94, 'character', 0.9905427098274231),
 (96, 97, 'actor', 0.9347705245018005),
 (99, 100, 'character', 0.8545386791229248)]

In [15]:
import bentoml
runner = bentoml.onnx.get("ner:latest").to_runner()
runner.init_local()

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.


In [21]:
output = data_process.decode(raw_batch["tokens"], raw_batch["id_to_classes"], torch.tensor(output))

In [22]:
output

[[(12, 12, 'character', 0.9745467305183411),
  (33, 33, 'location', 0.9113682508468628),
  (38, 42, 'date', 0.7391266226768494),
  (51, 51, 'date', 0.5650821924209595),
  (54, 54, 'location', 0.9242361187934875),
  (56, 57, 'actor', 0.9268550276756287),
  (59, 59, 'character', 0.9865744709968567),
  (61, 62, 'actor', 0.8973000645637512),
  (64, 64, 'character', 0.7989766597747803),
  (80, 85, 'date', 0.9063624143600464),
  (87, 89, 'location', 0.9150590300559998),
  (91, 92, 'actor', 0.9332297444343567),
  (94, 94, 'character', 0.9905427098274231),
  (96, 97, 'actor', 0.9347705245018005),
  (99, 100, 'character', 0.8545386791229248)]]